### Carga de datos a tablas de dimension utilizando Data Flow

In [ ]:
En el punto 7 explicamos como realizar la carga (el movimiento) de datos en la tabla de hechos "SalesFact".
Ahora utilizaremos también DATAFLOWS para realizar el movimiento de datos hacia dos tablas de dimensión,
las tablas "DimProduct" y "DimCustomer".

En un mismo DATAFLOW realizamos el movimiento de datos para ambas tablas.

In [ ]:
Para el primer movimiento se busca realizar la siguiente Query, con el fin de mover datos a la tabla "DimCustomer":

  SELECT 
    ct.[CustomerID],
    ct.[StoreID],
    st.[BusinessEntityID],
    st.[Name] as StoreName
  FROM 
    [Sales].[Customer] as ct
  LEFT JOIN 
    [Sales].[Store] as st 
  ON 
    ct.[StoreID] = st.[BusinessEntityID]
  WHERE  
    st.[BusinessEntityID] IS NOT NULL

Esta es la tabla "DimCustomer" donde se moveran los datos:

CREATE TABLE [dbo].[DimCustomer](
	[CustomerID] [int] NOT NULL,
	[StoreID] [int] NOT NULL,
	[BusinessEntityID] [int] NOT NULL,
	[StoreName] varchar(50) NOT NULL
)    

                                  JOIN                      SELECT                 ALTER ROW                   SINK
 ________________          ___________________          _______________          _______________          _______________
|                |        |                   |        |               |        |               |        |               |   
| CustomerStream |--------| CustomerJOINStore |--------| SelectColumns |--------|  RemoveNULLs  |--------|  DimCustomer  |
|________________|        |___________________|        |_______________|        |_______________|        |_______________|     
 ________________              | 
|                |             | 
|  StoreStream   |-------------'
|________________|

Esto realizamos en la opción de ALTER ROW

<center><img src="https://i.postimg.cc/MpfMTgFJ/adf1.png"></center>

In [ ]:
Para el segundo movimiento se busca realizar la siguiente Query, con el fin de mover datos a la tabla "DimProduct":

SELECT 
    prod.[ProductID],
    prod.[Name] as ProductName,
    prod.[SafetyStockLevel],
    model.[ProductModelID],
    model.[Name] as ProductModelName,
    category.[ProductSubcategoryID],
    category.[Name] AS ProductSubCategoryName
FROM 
    [Production].[Product] prod
LEFT JOIN 
    [Production].[ProductModel] model 
ON 
    prod.[ProductModelID] = model.[ProductModelID]
LEFT JOIN 
    [Production].[ProductSubcategory] category 
ON 
    prod.[ProductSubcategoryID] = category.[ProductSubcategoryID]
WHERE 
    prod.[ProductModelID] IS NOT NULL


Esta es la tabla "DimProduct" donde se moveran los datos:

CREATE TABLE [dbo].[DimProduct](
	[ProductSK] [int] IDENTITY(1,1) NOT NULL,    -- Se agegó este campo
	[ProductID] [int] NOT NULL,
	[ProductModelID] [int] NOT NULL,
	[ProductSubcategoryID] [int] NOT NULL,
	[ProductName] varchar(50) NOT NULL,
	[SafetyStockLevel] [smallint] NOT NULL,
	[ProductModelName] varchar(50) NULL,
	[ProductSubCategoryName] varchar(50) NULL
)

                                  JOIN                      SELECT                 ALTER ROW                   SINK
 ________________          ___________________          _______________          _______________          _______________
|                |        |                   |        |               |        |               |        |               |   
| CustomerStream |--------| CustomerJOINStore |--------| SelectColumns |--------|  RemoveNULLs  |--------|  DimCustomer  |
|________________|        |___________________|        |_______________|        |_______________|        |_______________|     
 ________________              | 
|                |             | 
|  StoreStream   |-------------'
|________________|

                                      JOIN                              JOIN                            SELECT                        ALTER ROW                 SURROGATE KEY               SINK
 ________________          _________________________          ________________________          ______________________          _____________________          _______________          ______________
|                |        |                         |        |                        |        |                      |        |                     |        |               |        |              |     
| ProductStream  |--------| ProductJOINProductModel |--------| ProductJOINSubcategory |--------| SelectColumnsProduct |--------| RemoveNULLsProduct  |--------|  SurrogateKey |--------|  DimProduct  |
|________________|        |_________________________|        |________________________|        |______________________|        |_____________________|        |_______________|        |______________|      
 ____________________              |                                |    
|                    |             |                                |
| ProductModelStream |-------------'                                |
|____________________|                                              |
 __________________________                                         | 
|                          |                                        | 
| ProductSubCategoryStream |----------------------------------------'
|__________________________|


Esto realizamos en la opción de ALTER ROW

<center><img src="https://i.postimg.cc/rF6cLmrT/adf2.png"></center>

Esto realizamos en la opción de SURROGATE KEY

<center><img src="https://i.postimg.cc/d0G8hV9G/adf3.png"></center>

Además, debemos agregar esta nueva columna desde la última opción "DimProduct" en la pestaña MAPPING

<center><img src="https://i.postimg.cc/ZnJkGnwH/adf4.png"></center>

In [ ]:
Tenemos los valores para ProductSK. Ahora necesitamos el siguiente conjunto de filas se añadan a la tabla 
de dimensión. Queremos asegurarnos de que el ProductSK empiece desde donde quedó. No debe comenzar desde 
el valor de 1 otra vez. Así que lo primero que podemos hacer es obtener el valor máximo actual en la columna 
ProductSK y luego almacenarlo en la caché y luego podemos hacer referencia a ese valor utilizando la función 
de cache lookup (búsqueda de caché).
    
    
                                  JOIN                      SELECT                 ALTER ROW                   SINK
 ________________          ___________________          _______________          _______________          _______________
|                |        |                   |        |               |        |               |        |               |   
| CustomerStream |--------| CustomerJOINStore |--------| SelectColumns |--------|  RemoveNULLs  |--------|  DimCustomer  |
|________________|        |___________________|        |_______________|        |_______________|        |_______________|     
 ________________              | 
|                |             | 
|  StoreStream   |-------------'
|________________|

                                      JOIN                              JOIN                            SELECT                        ALTER ROW                 SURROGATE KEY             DERIVED COLUMN                 SINK
 ________________          _________________________          ________________________          ______________________          _____________________          _______________          __________________          ______________
|                |        |                         |        |                        |        |                      |        |                     |        |               |        |                  |        |              |      
| ProductStream  |--------| ProductJOINProductModel |--------| ProductJOINSubcategory |--------| SelectColumnsProduct |--------| RemoveNULLsProduct  |--------|  SurrogateKey |--------|  DerivedColumn1  |--------|  DimProduct  |
|________________|        |_________________________|        |________________________|        |______________________|        |_____________________|        |_______________|        |__________________|        |______________|      
 ____________________              |                                |    
|                    |             |                                |
| ProductModelStream |-------------'                                |
|____________________|                                              |
 __________________________                                         | 
|                          |                                        | 
| ProductSubCategoryStream |----------------------------------------'
|__________________________|              SINK
 _________________________          ________________                                    
|                         |        |                |                                      
| DimProductStreamSynapse |--------| ProductSKCache |
|_________________________|        |________________|


Generamos una nueva caja llamada "DimProductStreamSynapse" la cual tiene un dataset que estará apuntando a la tabla "DimProduct"

<center><img src="https://i.postimg.cc/2yBxz737/adf5.png"></center>

Y obtenemos el valor máximo actual en la columna ProductSK por medio de una Query

<center><img src="https://i.postimg.cc/mrJwKngg/adf6.png"></center>

Creamos una nueva caja "ProductSKCache" que va a almacenar en cache el valor máximo de la columna ProductSK

<center><img src="https://i.postimg.cc/Dw2b531K/adf7.png"></center>

Luego de la caja de SurrogateKey generamos una nueva caja que nos permitirá modificar la columna ProduckSK:

ProductSK + ProductSKCache#outputs()[1].ProductSK

<center><img src="https://i.postimg.cc/B6rGQsJL/adf8.png"></center>

En el caso de que la tabla "DimProduct" no tuviera ningún registro, para evitar que la columna ProductSK nos devuelva
valores NULL, realizamos la siguiente expresión:

ProductSK + (iifNull((ProductSKCache#outputs()[1].ProductSK),0))